# <p style="text-align: center;">FIT5196 Assignment 1 (Task 1)</p>

### Student Name : Audi Rivai
### Student ID       : 30399262

Last Updated: 13 August 2020

Environment: Python 3.7.0 and Jupyter notebook Libraries used
* re (for the regular expression, included in Anaconda Python 3.7)
* os 
* langid (to classify the language)
* BeautifulShop and xmltodict (for open the xml file)

The content of this assignment are:
1. Introduction
2. Import Libraries
3. Loading Data
4. Parsing Data
5. Find English Text
6. Export Data
7. Summary



## 1. Introduction
This FIT5196 Assignment extracts data from text format file named 30399262 that contain the date of information about Twitter respond and converts it into a readable file format (XML) which are 30399262.xml consecutively. The required tasks are the following:

1. Extract the raw text from a `.txt` file by File I/O and regular expression then store into variable.
2. Process the raw data by using Python Programming Language and several languages. 
3. Export the result into `.xml` and files.

More details for each task will be given in the following cells below.

## 2. Import Libraries
In this assignment, there are 3 libraries will be used to support the process. They are Re, os, and langid.

In [1]:
!pip install langid
import re
import os
import langid

## 3. Loading Data
In this part, the raw text file named `30399262` folder using os package.

In [2]:
cwd = os.getcwd()
# create a list of text file name
all_text_files = os.listdir("30399262/") # use os.listdir to open the directory inside the 30399262

print("There are", len(all_text_files), "text file")

There are 2411 text file


## 4. Parsing Data

### 4.1. Regular Expression

This section, the regular expression below will be used in order to capture the necessary word from the tasks

__Part 1__ : Capture id number in each tweets.
```python
r'"id":"(.*?)"'
```


The explanation of the Regular Expression:
- matches the characters "id":" literally (case sensitive)
- 1st Capturing Group (.*?)
    - .*? matches any character (except for line terminators)
    - *? Quantifier Matches between zero and unlimited times, as few times as possible, expanding as needed
- " matches the character " literally (case sensitive)

__Part 2__ : clean the id number if there are some string or punctuation.

```python
r'[^\w\s]'
```


The explanation of the Regular Expression:
- Match a single character not present in the list below
- \w matches any word character (equal to [a-zA-Z0-9_])
- \s matches any whitespace character (equal to [\r\n\t\f\v ])

__Part 3__ : Capture text in each tweets.


```python
r'"text":"(.*?)("},"|","|"},{)'
```


The explanation of the Regular Expression:
- "text":" matches the characters "text":" literally (case sensitive)
- 1st Capturing Group (.*?) list below:
    - .*? matches any character (except for line terminators)
    - , is matches the character , literally
- 2nd Capturing Group ("},"|","|"},{):
    - "}," matches the characters "}," literally (case sensitive)
    - "," matches the characters "," literally (case sensitive)
    - "},{ matches the characters "},{ literally (case sensitive)

__Part 4__ : Capture date tweet in each tweets.
    
```python
r'"created_at":"(.*?)T'
```


The explanation of the Regular Expression:
- since all string date ends with T
- "created_at":" matches the characters "created_at":" literally (case sensitive)
- 1st Capturing Group (.*?)
    - .*? matches any character (except for line terminators)
    - *? Quantifier — Matches between zero and unlimited times, as few times as possible, expanding as needed
- T matches the character T literally (case sensitive)

In [3]:
%%time
# create an empty list for each information that we want such as text data is for all the text that have been open
data_list = []
# empty file for id list
id_list = []
# empty file for date list
date_list = []
# empty file for twitter list
twitter_list = []

# create loop for each text data in file that need to be open and append the text into the data_list
for data in range(len(all_text_files)):
    file_open = '30399262/'+all_text_files[data]
    with open(file_open,'r', encoding='utf-8') as infile:
        text = infile.read()
        data_list.append(text)
    
# apply the regular expression from part 1 and 2 for id number
    pattern_id = re.compile(r'"id":"(.*?)"') # create a pattern 
    matches = re.finditer(pattern_id, data_list[data]) # find the matching pattern from the data_list
# extract the id number from group 1 then delete some string such letter and punctuation
    for m in matches:
# then append the result to the id_list
        id_list.append(re.sub(r'[^\w\s]', '', m.group(1)))
    
# apply the regular expression from part 3 and 4 for twitter text
    pattern_tweets = re.compile(r'"text":"(.*?)("},"|","|"},{)') # create a pattern
    for match in re.finditer(pattern_tweets, data_list[data]): # find the matching pattern from the data_list
        twitter_text = (match.group(1))
# append the result to twitter_list
        twitter_list.append(twitter_text)
# apply the regular expression from part 5 for date 
    pattern = re.compile(r'"created_at":"(.*?)T') # create a pattern
    for match in re.finditer(pattern, data_list[data]): # find the matching pattern from the data_list
# extract the date from group 1 and add some "
        date = (match.group(1))
# append the date to the date list
        date_list.append('"'+date+'"')
    
print(len(twitter_list))
print(len(id_list))

213134
213134
Wall time: 17.4 s


## 5. Find English Text

In this section we will be looking which text is written in english and remove other languages using [function](https://github.com/saffsd/langid.py) `langid.classify()` we will be looking the result that contain english sentence. we will looping the twitter text and add into and empty list 

In [4]:
%%time
# set time, it may take a while

# create empty list for id, twitter text and date
en_twitter_list = []
en_id_list = []
en_date = []

# run a loop that get the englist ID twitter text and date using langid.classify
for i in range(len(twitter_list)):
    text = twitter_list[i]
    # encode and decode the text into a utf-16 using eval
    if "\\u" in text:
        cleaner_text = eval("text.encode('utf_8').decode('unicode_escape')")\
                        .encode('utf-16', 'surrogatepass').decode('utf-16')
    else:
        cleaner_text = text
    # find the english fucntion
    if langid.classify(cleaner_text)[0] == "en":
        en_twitter_list.append(twitter_list[i])
        en_id_list.append(id_list[i])
        en_date.append(date_list[i])
        
print("Number of Tweets: ", len(en_twitter_list))
print("Number of Tweets: ", len(en_id_list))

Number of Tweets:  120415
Number of Tweets:  120415
Wall time: 8min


After we know that there are plenty of tweets we need to clean the just to make sure the word is prepare to write in xml. Some words still contain with double qoutes [HTML](https://www.rapidtables.com/web/html/html-codes/htm-code-quote.html) decode. In this case we will be looking `&quot;` in html this is a ". Therefore we need to change them into on quotes

In [5]:
clean_text = []
# clean all the double backslash by using replace
for text in range(len(en_twitter_list)):
    # in the html single quotes in html is &quot; therefore we may replace this
    new_text = en_twitter_list[text].replace("\\n", "\n")\
                .replace('\\"', '&quot;').replace("\\r", "")
    clean_text.append(new_text)

Next we will be put all the value into a dictionary.

In [6]:
# create dictionary for each date

dct = list(zip(en_id_list, clean_text))
dcti = {}
for i, j in zip(en_date, dct):
    dcti.setdefault(i, []).append(j)

## 6. Export Data

since we aleady fill all he value into the dictioary, in this section we will be export the data into the xml file 

In [7]:
# write the xml file that contain from the dictionary
xml= open("30399262.xml","w+", encoding="utf-8-sig")
xml.write("<"+'?xml version="1.0" encoding="UTF-8"?'+">\n")
xml.write("<"+'data'+">\n")
for tweet_dates, value in dcti.items():
    xml.write("<"+'tweets date='+tweet_dates+">\n")
    for i in value:
        tweets = i[1]
        # use surrogatepass to print the emoji when they found an unicode
        if "\\u" in tweets: 
            clean_tweets = eval("tweets.encode('utf_8').decode('unicode_escape')")\
            .encode('utf-16', 'surrogatepass').decode('utf-16')
        else:
            clean_tweets = tweets
        # clear the remain backslash
        clean_tweets = clean_tweets.replace("\\", "")
        xml.write("<"+'tweet id='+'"'+i[0]+'"'+">"+clean_tweets+"</"+"tweet"+">\n")
    xml.write("</"+'tweets'+">\n")
xml.write("</"+'data'+">")
xml.close()

In order to make sure that the xml file can be open we will be using 2 packages which is xmltodict and BeautifulSoup. The tools is to make sure that the file can be open.

In [8]:
# import xmltodict

In [9]:
# with open('30399262.xml', encoding="UTF-8") as fd:
#     doc = xmltodict.parse(fd.read())

# doc

We can see that the xml file can be seen perfectly. next we will be looking the xml using BeautifulSoup. Before we start using the BeautifulSoup there are some step that need to be done. since the xml data is large there is a configuration that need to be change in the jupyter notebook:
* Open the anaconda command prompt
* Type this codes:
    * `jupyter notebook --generate-config`
    * `jupyter notebook --NotebookApp.iopub_data_rate_limit=10e10`

This code will reopen a new jupyter notebook that can open big memory file. After that you need to run the cells all over again and then start with the BeautifulSoup

In [10]:
# from bs4 import BeautifulSoup

In [11]:
# read the xml file
# soup = open('30399262.xml', "r", encoding="utf-8")
# content = soup.read()
# soup = BeautifulSoup(content, 'lxml')

In [12]:
# find the data
# soup.find('tweets')

# 7. Summary

This assignment is help to understand of parsing the raw text file such as txt into readable file formats such as xml, json, etc. The regular expression is used to extract the text file. 
- **Tokenization, collocation extraction**. By using the `re` package, capturing text from text in order to make readable. 
- **Parsing**, Python Programming Language will be used for processing/parsing the raw text. 
- **Exporting data into specific format**. Basic Python File I/O is used to export the result into XML File Format

## 8. References
- Regex101. (2020, August 26). *Regular Expression 101* Retrieved from https://regex101.com/
- MattDMo. (2016, July 1). *How to work with surrogate pairs in Python?*. Retrieved from http://stackoverflow.com/a/4739636
- RapidTables. (2020, August 23) *HTML code for quotes*. Retrieved from https://www.rapidtables.com/web/html/html-codes/htm-code-quote.html
- saffsd. (2017, July 15) *langid.py readme*. Retrieved from https://github.com/saffsd/langid.py
- Python Software Foundation. (2020, August 23) *Regular expression operations* https://docs.python.org/2/library/re.html
- Dan. (2010, February 6) *Counting the Number of keywords in a dictionary in python* https://stackoverflow.com/questions/2212433/counting-the-number-of-keywords-in-a-dictionary-in-python#:~:text=The%20number%20of%20distinct%20words,using%20the%20len()%20function.&text=To%20get%20all%20the%20distinct,keys()%20method.